### **README**

# What this does

This script allows you to download any number of **Dashboards** into a singular Excel file. Each Tile's query will have row limits removed. You can also specify a file name for the download. 

Note: **Be careful** , this can create large Excel files depending on the amount of data returned by each query. It's recommended that you modify this script to download each tile as a seperate CSV if an unlimited download of any one tile will return a large amount of data. 

# Notes
`DownloadDashboardUnlimited(dash_id, file_name)` accepts two inputs:

- `dash_id`(string) - accepts a dashboard ID.
- `file_name`(string) - the desired output file name of the final excel file.

If using colab version, you will need to input your api url, client_id, and client_secret in the `## colab inputs ##` section. 

```
os.environ['LOOKERSDK_BASE_URL'] = 'https://example.looker.com'
os.environ['LOOKERSDK_API_VERSION'] = '3.1'
os.environ['LOOKERSDK_CLIENT_ID'] = 'client id here'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'client secret here'
```

If using python (local) version, these values should be set for the sdk per either an `.ini` file or as an environment variable. See Python SDK documentation for more information. https://pypi.org/project/looker-sdk/

In [2]:
### colab specific ###
# install the sdk
!pip install looker-sdk
# set environ variables
import os

## colab inputs ##
os.environ['LOOKERSDK_BASE_URL'] = 'https://dcl.api.dev.looker.com'
os.environ['LOOKERSDK_API_VERSION'] = '3.1'
os.environ['LOOKERSDK_CLIENT_ID'] = 'vY3cbCbGrPdCjsjr8CnF'
os.environ['LOOKERSDK_CLIENT_SECRET'] = 'pB3BTTgb9m3bQz23p566FWsR'

# will need these for downloading the file from colab
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import files
from oauth2client.client import GoogleCredentials
# auth stuff for download
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
### import packages ###
# import and init
import looker_sdk
sdk = looker_sdk.init31("looker.ini")

import pandas as pd
import csv
import json
import re

In [38]:
## inputs ##

dash_id = '1317'
file_name = 'blech'


In [41]:
def DownloadDashboardUnlimited(dash_id, file_name):

  data = sdk.search_dashboard_elements(dashboard_id=dash_id)
  numelemnets = len(data)

  with pd.ExcelWriter(file_name+'.xlsx') as writer:

    for x in range(numelemnets):
      element_query_body = data[x].result_maker.query
      element_query_body['limit'] = -1
      elements_needed = ['model','view','fields','filters','sorts','limit','query_timezone']
      listlen = len(elements_needed)

      final_body = {}

      for y in range(listlen):
        try:
          new_el = element_query_body[elements_needed[y]]
        except:
          i+=1
        else:
          final_body[elements_needed[y]] = new_el
      
      resultset = sdk.run_inline_query('csv', final_body)
  
      with open('temp.csv', 'w', newline='') as file:
            file.write(resultset)
      df = pd.read_table('temp.csv', delimiter =",",engine='python')
      df.to_excel(writer, sheet_name=('Sheet ' + str(x)))

   Orders Created Year  Orders Count
0                 2019         13657
1                 2018         10752
2                 2017          6248
3                 2016          1600
   Orders User ID  Orders Count
0              73            25
1            3454            22
2            1729            21
3             612            21
4            3300            21
  Orders Status  Orders Count
0      complete         31377
1       pending           505
2     cancelled           375


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
DownloadDashboardUnlimited(dash_id, file_name)

## colab stuff
from google.colab import files 
files.download(file_name+'.xlsx')